In [ ]:
# load the libraries that we'll use  
from mutagen.mp3 import MP3  
from mutagen.easyid3 import EasyID3  
# import mutagen.id3  
# from mutagen.id3 import ID3, TIT2, TIT3, TALB, TPE1, TRCK, TYER 
import glob  
import numpy as np 
import os
import json
import csv

In [ ]:
class TagEditor:
    album_path = ""
    album_list = []
    
    def __init__(self,album_path,genre,org):
        self.album_path = album_path
        self.genre = genre
        self.org = org
    
    def updateAlbumList(self):
        with open("data.csv", "r") as file:
            reader = csv.reader(file)
            for row in reader:
                self.album_list.append(row)
    
    def renameFolder(self,old,newName):
        new = old.split("\\")
        new[-1] = newName
        os.rename(old,"\\".join(new))
      
    def getAlbums(self):
        count = 0
        for album in os.listdir(self.album_path):
            glob_filez = glob.glob(self.album_path + album + "\\*.mp3")
            self.mp3TagEditor(glob_filez,count)
            album_abs_path = os.path.abspath(os.path.join(album_path, album))
            self.renameFolder(album_abs_path,self.album_list[count][0])
            count += 1
        
    def mp3TagEditor(self,glob_filez, count):
        for i in np.arange(0, len (glob_filez)):
            song = glob_filez[i]  
            # turn it into an mp3 object using the mutagen library  
            mp3file = MP3(song, ID3=EasyID3) 
            
            mp3file['organization'] = [self.org]
            mp3file['genre'] = self.genre
            mp3file['tracknumber'] =  str(i+1) + '/' + str(len(glob_filez))  
            
            mp3file['album'] = [self.album_list[count][0]]
            mp3file['albumartist'] = mp3file['composer'] = [self.album_list[count][1]]
            mp3file['date'] = [self.album_list[count][-1]] 
            
            # mp3file['title'] = [glob_filez[i].split("\\")[-1]]
            # mp3file['artist'] = ['Pawan Kalyan, Trisha Krishnan, Kruthi Kharbanda']
            
            # mp3file['encodedby'] = ['']
            # mp3file['copyright'] = ['http://iSongs.info']
            # save the changes that we've made
            mp3file.save()
            # os.rename(glob_filez[i],glob_filez[i].replace("-SenSongsMp3.Co",""))

In [ ]:
# driver code 
if __name__ == '__main__':
    # extract the file names (including folders)  
    # for the mp3s in the album  
    album_path = "C:\\Users\\in the presence\\Documents\\MyWork\\mp3-tag-editor\\test ablums\\"
    genre = "Telugu"
    org = "DSA"
    tagObj = TagEditor(album_path,genre,org)
    tagObj.updateAlbumList()  
    tagObj.getAlbums()  




In [ ]:
album_path = "C:\\Users\\in the presence\\Documents\\MyWork\\mp3-tag-editor\\test ablums\\"
for album in os.listdir(album_path):
    glob_filez = glob.glob(album_path+album+"\\*.mp3")
    break

In [ ]:

 # print the first element of filez:  
# filez[1]  
print(*glob_filez, sep="\n")

In [ ]:
# we want to extract the track number from the file name  
# the file name is the last part of the string in 'filez'  
# the different parts of 'filez' are split with a   
# forward slash (/)  
# let's split this element up and see what we get  
glob_filez[0].split("\\")

In [ ]:
# check the ID3 tags for the mp3s in our folder  
for i in np.arange(0, len(glob_filez)):  
	song = glob_filez[i]  
	mp3file = MP3(song, ID3=EasyID3)  
	print(mp3file["album"][0])
	print(any(x in mp3file["album"][0].split(" ") for x in album_list))
	print(glob_filez[i].split("\\")[-1])
	break

In [ ]:

def updatemp3tags(mp3file, data):
    try:
        for key, value in data.items():
            if mp3file[key] and mp3file[key][0] not in value:
                value.append(mp3file[key][0])
                data[key] = value
    except Exception as e:
        print("In exeption")
        print(e)   
    return data
    

In [ ]:
# Load the JSON file
with open("mp3tags.json", "r") as file:
    data = json.load(file)
            
# test_album_path = "C:\\Users\\in the presence\\Documents\\MyWork\\mp3-tag-editor\\test ablums\\"
test_album_path = "C:\\Users\\kOUNDINYA\\Music\\Aditya\\Telugu\\"
for album in os.listdir(test_album_path):
    glob_filez = glob.glob(test_album_path+album+"\\*.mp3")
    for i in np.arange(0, len(glob_filez)):  
        song = glob_filez[i]  
        mp3file = MP3(song, ID3=EasyID3) 
        try:
            data = updatemp3tags(mp3file,data)
        except Exception as e:
            pass
            # print("In exeption")
            # print(e)
    
# Write the modified data back to the file
with open("mp3tags.json", "w") as file:
    json.dump(data, file, indent=4)


In [ ]:
# test_album_path = "C:\\Users\\kOUNDINYA\\Music\\Aditya\\Telugu\\"
test_album_path = "C:\\Users\\in the presence\\Documents\\MyWork\\mp3-tag-editor\\test ablums"

result = []
for album in os.listdir(test_album_path):
    result.append(album)

with open("output.txt", "w") as file:
    output = "\n".join(result)
    file.write(output)

In [ ]:
old = "C:\\Users\\in the presence\\Documents\\MyWork\\mp3-tag-editor\\test ablums"
new = old.split("\\")
new[-1] = "test ablums"
print()
os.rename(old,"\\".join(new))

In [ ]:
album_path = "C:\\Users\\in the presence\\Documents\\MyWork\\mp3-tag-editor\\test ablums"
for album in os.listdir(album_path):
    album_abs_path = os.path.abspath(os.path.join(album_path, album))
    print(album_abs_path)

In [9]:
import os
import shutil
from mutagen.easyid3 import EasyID3
from mutagen.mp3 import MP3
from mutagen.id3 import APIC, ID3
import mimetypes

In [2]:
def print_mp3_metadata(mp3_file_path):
    # Load the MP3 file
    audio = MP3(mp3_file_path, ID3=EasyID3)

    # Print all available metadata properties
    print("Available metadata properties:")
    for key in audio.keys():
        print(f"{key}: {audio[key]}")

# Example usage:
mp3_file_path = "D:\\utilities\\mp3-tag-editor\\updated_Akon\\Beautifull.mp3"
print_mp3_metadata(mp3_file_path)

Available metadata properties:
album: ['Freedom']
title: ['Beautiful']
artist: ['Akon']
tracknumber: ['9']
date: ['2013']


In [19]:

def update_mp3_metadata(album_path, output_path, metadata={}):
    # If output path is not provided, create a default output directory
    if output_path == '':
        output_path = os.path.join(os.getcwd(), f"updated_{os.path.basename(album_path)}")
        os.makedirs(output_path, exist_ok=True)

    # Get a list of all MP3 files in the album path
    mp3_files = [file for file in os.listdir(album_path) if file.endswith(".mp3")]

    # Counter to track the order of files
    track_number = 1

    for mp3_file in mp3_files:
        # Read the MP3 file
        file_path = os.path.join(album_path, mp3_file)
        audio = MP3(file_path, ID3=EasyID3)

        # Create a new MP3 file with the same audio data
        new_file_path = os.path.join(output_path, mp3_file)
        shutil.copy2(file_path, new_file_path)
        new_audio = MP3(new_file_path, ID3=EasyID3)

        # Update MP3 properties if available
        if metadata:
            for key, value in metadata.items():
                if value:
                    new_audio[key] = value
                

        # Update track number
        new_audio['tracknumber'] = str(track_number)

        # Save the changes
        new_audio.save()

        track_number += 1

        print(f"Updated properties for {mp3_file} and saved to {new_file_path}")



In [20]:
album_path = 'D:\\utilities\\mp3-tag-editor\\testalbums\\Akon'
output_path = ''

# Example usage:
metadata = {
    'album': '',
    # 'encodedby': 'test',
    'title': 'Beautiful',
    'version': 'test',
    'artist': 'test',
    'albumartist': 'test',
    'genre': 'test',
    'date': '2023',
    'website': 'test',
}

update_mp3_metadata(album_path, output_path, metadata)

Updated properties for Beautifull.mp3 and saved to d:\utilities\mp3-tag-editor\updated_Akon\Beautifull.mp3
Updated properties for Holla Holla.mp3 and saved to d:\utilities\mp3-tag-editor\updated_Akon\Holla Holla.mp3
Updated properties for Keep Up.mp3 and saved to d:\utilities\mp3-tag-editor\updated_Akon\Keep Up.mp3
Updated properties for Keep You Much Longer.mp3 and saved to d:\utilities\mp3-tag-editor\updated_Akon\Keep You Much Longer.mp3
Updated properties for Party Animal.mp3 and saved to d:\utilities\mp3-tag-editor\updated_Akon\Party Animal.mp3
Updated properties for Right now.mp3 and saved to d:\utilities\mp3-tag-editor\updated_Akon\Right now.mp3
Updated properties for Troublemaker.mp3 and saved to d:\utilities\mp3-tag-editor\updated_Akon\Troublemaker.mp3
